In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('myData_filtered.csv')
df.head()

,Unnamed: 0,nobs,mean,skew,kurtosis,median,mode,std,low,peak,q25,q75,iqr,user_name,sample_date,age_range,pronunciation,gender
0,0,85,148.411765,-0.170872,-1.218425,140.0,90.0,67.576587,25.0,260.0,90.0,205.0,115.0,1028,20100710,Adult,American,Male
1,1,103,186.116505,-1.241716,0.065125,210.0,220.0,76.691071,25.0,270.0,185.0,232.5,47.5,1337ad,20170321,Adult,Other,Female
2,2,117,192.350427,-1.402413,0.727092,210.0,200.0,67.433325,25.0,265.0,190.0,235.0,45.0,1337ad,20170321,Adult,Other,Female
3,3,119,157.661064,-0.532878,-1.508620,205.0,30.0,92.220587,25.0,270.0,40.0,227.5,187.5,1337ad,20170321,Unknown,Other,not_specified
4,4,34,159.558824,-0.893693,-0.636448,185.0,25.0,73.748057,25.0,245.0,122.5,215.0,92.5,1snoke,20120412,Adult,American,Male


In [3]:
pd.value_counts(df.gender)


Male             4725
Female            379
not_specified      58
Name: gender, dtype: int64

In [4]:
temp_df = df[df.gender != 'not_specified']

In [5]:
pd.value_counts(temp_df.gender)


Male      4725
Female     379
Name: gender, dtype: int64

In [6]:
pd.value_counts(temp_df.pronunciation)

American      2454
Other          890
European       713
British        684
Indian         213
Australian     150
Name: pronunciation, dtype: int64

In [7]:
pd.value_counts(temp_df.age_range)

Adult      4196
Youth       586
Unknown     210
Senior      112
Name: age_range, dtype: int64

In [8]:
temp_df_1 = temp_df[temp_df.age_range != 'Unknown']

In [9]:
temp_df_1 = temp_df_1[temp_df_1.age_range != 'Senior']

In [10]:
pd.value_counts(temp_df_1.age_range)

Adult    4196
Youth     586
Name: age_range, dtype: int64

In [11]:
temp_df_1.shape

(4782, 18)

In [12]:
temp_df_1.drop(['Unnamed: 0'],inplace=True,axis=1)

In [13]:
temp_df_1.head(15)

,nobs,mean,skew,kurtosis,median,mode,std,low,peak,q25,q75,iqr,user_name,sample_date,age_range,pronunciation,gender
0,85,148.411765,-0.170872,-1.218425,140.0,90.0,67.576587,25.000000,260.0,90.00,205.0,115.00,1028,20100710,Adult,American,Male
1,103,186.116505,-1.241716,0.065125,210.0,220.0,76.691071,25.000000,270.0,185.00,232.5,47.50,1337ad,20170321,Adult,Other,Female
2,117,192.350427,-1.402413,0.727092,210.0,200.0,67.433325,25.000000,265.0,190.00,235.0,45.00,1337ad,20170321,Adult,Other,Female
4,34,159.558824,-0.893693,-0.636448,185.0,25.0,73.748057,25.000000,245.0,122.50,215.0,92.50,1snoke,20120412,Adult,American,Male
5,200,147.478333,0.266550,-1.097457,135.0,100.0,74.874091,25.000000,275.0,100.00,220.0,120.00,23yipikaye,20100807,Adult,Other,Male
6,84,149.869048,-0.150714,-1.130314,170.0,80.0,55.081061,24.000000,250.0,95.00,190.0,95.00,314piwm,20130617,Youth,American,Male
7,97,113.195876,0.209949,3.253056,120.0,130.0,43.696384,25.000000,265.0,110.00,130.0,20.00,AJN,20131002,Adult,Other,Male
8,83,117.650602,1.005635,0.975156,105.0,85.0,44.378828,25.000000,240.0,90.00,125.0,35.00,AT,20130718,Adult,British,Male
9,82,173.956520,-0.429759,-0.885734,205.0,120.0,56.458512,22.038567,265.0,120.00,220.0,100.00,Aaron,20080318,Adult,Other,Male
10,101,158.514851,0.190964,-0.997187,130.0,115.0,55.260609,25.000000,270.0,120.00,210.0,90.00,Aaron,20080318,Adult,Other,Male


In [ ]:
##CHECKPOINT-1##
'''so the data labels are highly squeued at this point'''
'''i will normalise it now and train the squeued data and see its accuracy'''
'''if the accuracy is good then will leave it that way or we will use data augmentation to create new data'''

'if the accuracy is good then will leave it that way or we will use data augmentation to create new data'

In [14]:
temp_df_1.isna().sum()

nobs             0
mean             0
skew             0
kurtosis         0
median           0
mode             0
std              0
low              0
peak             0
q25              0
q75              0
iqr              0
user_name        2
sample_date      0
age_range        0
pronunciation    0
gender           0
dtype: int64

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
features = temp_df_1.iloc[:,:-5]
labels = temp_df_1.iloc[:,-1]

In [17]:
features.head()

,nobs,mean,skew,kurtosis,median,mode,std,low,peak,q25,q75,iqr
0,85,148.411765,-0.170872,-1.218425,140.0,90.0,67.576587,25.0,260.0,90.0,205.0,115.0
1,103,186.116505,-1.241716,0.065125,210.0,220.0,76.691071,25.0,270.0,185.0,232.5,47.5
2,117,192.350427,-1.402413,0.727092,210.0,200.0,67.433325,25.0,265.0,190.0,235.0,45.0
4,34,159.558824,-0.893693,-0.636448,185.0,25.0,73.748057,25.0,245.0,122.5,215.0,92.5
5,200,147.478333,0.266550,-1.097457,135.0,100.0,74.874091,25.0,275.0,100.0,220.0,120.0


In [18]:
labels.head()

0      Male
1    Female
2    Female
4      Male
5      Male
Name: gender, dtype: object

In [175]:
scale = StandardScaler()
scale.fit(features)
scaled_temp_df_1 = scale.transform(features)

In [176]:
import pickle

In [179]:
pickle.dump(scale, open('scaler.sav', 'wb'))

In [181]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [183]:
from sklearn.svm import SVC

In [182]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(scaled_temp_df_1,labels,stratify=labels)

In [184]:
from sklearn.model_selection import GridSearchCV

In [188]:
param_grid = {'C': [1.25],  
              'gamma': [1.3]
             }  
svc = SVC()
GS = GridSearchCV(svc,param_grid,n_jobs=-1)

In [189]:
GS.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1.25], 'gamma': [1.3]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [190]:
GS.best_score_

0.939487884756588

In [191]:
GS.best_estimator_

SVC(C=1.25, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.3, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [192]:
from sklearn.metrics import accuracy_score

In [193]:
accuracy_score(GS.predict(X_test),y_test)

0.9389632107023411

In [194]:
from sklearn.metrics import classification_report

In [195]:
print(classification_report(y_test,GS.predict(X_test)))

              precision    recall  f1-score   support

           0       0.75      0.24      0.37        87
           1       0.94      0.99      0.97      1109

    accuracy                           0.94      1196
   macro avg       0.85      0.62      0.67      1196
weighted avg       0.93      0.94      0.92      1196



In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.73      0.41      0.53        87
           1       0.96      0.99      0.97      1109

    accuracy                           0.95      1196
   macro avg       0.85      0.70      0.75      1196
weighted avg       0.94      0.95      0.94      1196



In [32]:
from sklearn.metrics import roc_auc_score,roc_curve

In [33]:
svc_prob = SVC(probability=True,C=1.25,gamma=1.3)
svc_prob.fit(X_train,y_train)
prob_pred = svc_prob.predict_proba(X_test)[:,0]

In [34]:
y_pred = (svc_prob.predict_proba(X_test)[:,1] >= 0.65) 

0.003137842524416568

In [35]:
fpr,tpr,threshold= roc_curve(y_test,prob_pred)

In [ ]:
threshold

array([1.99575479, 0.99575479, 0.98214041, 0.98160643, 0.98144985,
       0.98134118, 0.97981024, 0.9798092 , 0.97821745, 0.97821635,
       0.97662314, 0.97661586, 0.9683991 , 0.9682937 , 0.96718632,
       0.96715104, 0.96446328, 0.96442606, 0.96372206, 0.96371018,
       0.96158395, 0.96146326, 0.9590943 , 0.95904584, 0.95808262,
       0.95803543, 0.95726348, 0.95725917, 0.95703356, 0.95699146,
       0.95570553, 0.95566723, 0.95498831, 0.95498403, 0.95471112,
       0.95468806, 0.95450147, 0.95449579, 0.95357952, 0.9535704 ,
       0.95314281, 0.95300774, 0.95239343, 0.95234327, 0.95130737,
       0.95128006, 0.9512494 , 0.9512274 , 0.94997705, 0.94995021,
       0.94687721, 0.94683   , 0.94473341, 0.94461417, 0.93774783,
       0.93769213, 0.93393402, 0.93374688, 0.93018683, 0.93004128,
       0.92736105, 0.92686588, 0.92443179, 0.92376792, 0.91828676,
       0.91783682, 0.90017505, 0.89978862, 0.89860369, 0.89847871,
       0.89451795, 0.88798715, 0.88671479, 0.88358725, 0.86542

In [196]:
from sklearn.metrics import recall_score,precision_score,f1_score

In [37]:
precision = []
recall = []
f1 = []
for i in threshold:
  y_pred_thresh = np.where(prob_pred > i ,0,1)
  precision.append(precision_score(y_test,y_pred_thresh))
  recall.append(recall_score(y_test,y_pred_thresh))
  f1.append(f1_score(y_test,y_pred_thresh))
prec_recall_thresh_series = pd.concat([pd.Series(precision),pd.Series(recall),pd.Series(threshold),pd.Series(f1)],axis=1)
prec_recall_thresh_series.columns = ['precision','recall','threshold','f1']
print(prec_recall_thresh_series.sort_values(by='f1',ascending=False))

#0.5769
#0.4589


     precision    recall  threshold        f1
16    0.947549  0.993688   0.699439  0.970070
17    0.948320  0.992786   0.604811  0.970044
19    0.949094  0.991885   0.500000  0.970018
18    0.948276  0.991885   0.540363  0.969590
15    0.945159  0.994590   0.783187  0.969244
..         ...       ...        ...       ...
117   0.945946  0.031560   0.015554  0.061082
118   0.944444  0.030658   0.015471  0.059389
119   0.937500  0.013526   0.013482  0.026667
120   0.933333  0.012624   0.013323  0.024911
121   1.000000  0.000902   0.005487  0.001802

[122 rows x 4 columns]


TRying smote on male/female classifier

In [197]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=10)
Aug_scaled_features_mf,new_labels_mf=sm.fit_sample(scaled_temp_df_1,labels)
print(pd.value_counts(new_labels_mf))

1    4434
0    4434
dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [198]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Aug_scaled_features_mf,new_labels_mf,stratify=new_labels_mf)

In [199]:
param_grid = {'C': [10],  
              'gamma': [3]
             }

svc = SVC()
GS_smote = GridSearchCV(svc,param_grid,n_jobs=-1)

In [200]:
GS_smote.fit(X_train,y_train)
print(GS_smote.best_params_)
print(GS_smote.best_score_)

{'C': 10, 'gamma': 3}
0.9682749699191631


In [201]:
print(classification_report(y_test,GS_smote.predict(X_test)))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1109
           1       0.99      0.96      0.98      1108

    accuracy                           0.98      2217
   macro avg       0.98      0.98      0.98      2217
weighted avg       0.98      0.98      0.98      2217



In [202]:
roc_auc_score(y_test,GS_smote.predict(X_test))

0.9756370587871468

In [203]:
pickle.dump(GS_smote, open('male_female.sav', 'wb'))

In [ ]:
###FIRST CLASSIFIER DONE.###

In [205]:
labels_1 = temp_df_1.iloc[:,-2]

In [206]:
labels_1

0       American
1          Other
2          Other
4       American
5          Other
          ...   
5157    American
5158    American
5159    American
5160    American
5161    American
Name: pronunciation, Length: 4782, dtype: object

In [ ]:
'''from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(scaled_temp_df_1,labels_1,stratify=labels)'''

'from sklearn.model_selection import train_test_split\nX_train,X_test,y_train,y_test = train_test_split(scaled_temp_df_1,labels_1,stratify=labels)'

In [ ]:
'''param_grid_1 = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]
              }   
svc_1 = SVC()
GS_1 = GridSearchCV(svc_1,param_grid_1,n_jobs=-1)'''

"param_grid_1 = {'C': [0.1, 1, 10, 100, 1000],  \n              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]\n              }   \nsvc_1 = SVC()\nGS_1 = GridSearchCV(svc_1,param_grid_1,n_jobs=-1)"

In [ ]:
'''GS_1.fit(X_train,y_train)'''

'GS_1.fit(X_train,y_train)'

In [ ]:
'''print(GS_1.best_score_)
print(GS_1.best_estimator_)'''

'print(GS_1.best_score_)\nprint(GS_1.best_estimator_)'

In [ ]:
'''accuracy_score(GS_1.predict(X_test),y_test)'''

'accuracy_score(GS_1.predict(X_test),y_test)'

In [ ]:
'''GS_1.predict([X_test[0]])'''

'GS_1.predict([X_test[0]])'

In [207]:
pd.value_counts(labels_1)

American      2304
Other          810
European       705
British        607
Indian         209
Australian     147
Name: pronunciation, dtype: int64

In [ ]:
'''from sklearn.ensemble import RandomForestClassifier
'''

'from sklearn.ensemble import RandomForestClassifier\n'

In [208]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=10)
Aug_scaled_features_accent,new_labels_accent=sm.fit_sample(scaled_temp_df_1,labels_1)
print(pd.value_counts(new_labels_accent))

American      2304
British       2304
European      2304
Indian        2304
Australian    2304
Other         2304
dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [219]:
new_labels_accent = le.fit_transform(new_labels_accent)

In [220]:
X_train,X_test,y_train,y_test = train_test_split(Aug_scaled_features_accent,new_labels_accent,stratify=new_labels_accent)

In [ ]:
'''rf = RandomForestClassifier()
rd_params = {
 'max_features': ['auto', 'sqrt'],
 'n_estimators': [200, 400, 600, 800]
}
'''
'''
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

'''

"\n{'bootstrap': [True, False],\n 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],\n 'max_features': ['auto', 'sqrt'],\n 'min_samples_leaf': [1, 2, 4],\n 'min_samples_split': [2, 5, 10],\n 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}\n\n"

In [ ]:
'''GS_rf = GridSearchCV(rf,rd_params,n_jobs=-1)
GS_rf.fit(X_train,y_train)'''

'GS_rf = GridSearchCV(rf,rd_params,n_jobs=-1)\nGS_rf.fit(X_train,y_train)'

In [221]:
param_grid_2 = {'C': [993],  
              'gamma': [2.8]
              }  
svc_2 = SVC()
GS_2 = GridSearchCV(svc_2,param_grid_2,cv=10,n_jobs=-1,verbose=True)

In [222]:
GS_2.fit(X_train,y_train)
print(GS_2.best_score_)
print(GS_2.best_params_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   57.7s finished


0.8693105110896818
{'C': 993, 'gamma': 2.8}


{'C': 985, 'gamma': 1.7} ---- 0.848

{'C': 989, 'gamma': 1.9} ---- 0.8488

{'C': 992, 'gamma': 2.2} ---- 0.8497

{'C': 993, 'gamma': 2.8} ---- 0.8534

In [223]:
accuracy_score(y_test,GS_2.predict(X_test))

0.8755787037037037

In [224]:
print(classification_report(y_test,GS_2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.74      0.66      0.69       576
           1       0.95      0.98      0.97       576
           2       0.90      0.93      0.91       576
           3       0.85      0.89      0.87       576
           4       0.94      0.96      0.95       576
           5       0.85      0.84      0.85       576

    accuracy                           0.88      3456
   macro avg       0.87      0.88      0.87      3456
weighted avg       0.87      0.88      0.87      3456



In [225]:
y_test[0]

4

In [226]:
pickle.dump(GS_2, open('accent.sav', 'wb'))

In [ ]:
'''print(GS_rf.best_score_)
print(GS_rf.best_estimator_)'''

'print(GS_rf.best_score_)\nprint(GS_rf.best_estimator_)'

In [104]:
from xgboost import XGBClassifier


In [141]:
XGB = XGBClassifier()
param_XGB = {
    
    'max_depth':[9,10,12],
    'n_estimators':[200,250],
    'learning_rate': [0.1,0.01,0.005],
    'booster':['dart']
}

In [218]:
y_train[0]

'Other'

In [142]:
GS_XGB = GridSearchCV(estimator=XGB,param_grid=param_XGB)
GS_XGB.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'booster': ['dart'],
                         'learning_rate': [0.1, 0.01, 0.005],
                         'max_depth': [9, 10, 12], 'n_estimators':

{'learning_rate': 0.1,
 'max_depth': 15,
 'n_estimators': 170,
 'objective': 'multi:softprob'} ----- 0.816

{'learning_rate': 0.12,
 'max_depth': 19,
 'n_estimators': 200,
 'objective': 'multi:softprob'} ----- 0.819

 {'learning_rate': 0.18,
 'max_depth': 21,
 'n_estimators': 230,
 'objective': 'multi:softprob'} ----- 0.821

 {'learning_rate': 0.18,
 'max_depth': 20,
 'n_estimators': 280,
 'objective': 'multi:softprob'} ----- 0.8214



In [144]:
GS_XGB.best_params_

{'booster': 'dart',
 'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 250}

In [146]:
GS_XGB.best_score_

0.8786952755018396

In [147]:
print(classification_report(y_test,GS_XGB.predict(X_test)))

              precision    recall  f1-score   support

       Adult       0.88      0.99      0.93      1049
       Youth       0.40      0.03      0.05       147

    accuracy                           0.88      1196
   macro avg       0.64      0.51      0.49      1196
weighted avg       0.82      0.88      0.82      1196



the classifier which we have used for predicting accent is also svm('C': 993,gamma: 2.8)

In [227]:
labels_2 = temp_df_1.iloc[:,-3]

In [228]:
pd.value_counts(labels_2)

Adult    4196
Youth     586
Name: age_range, dtype: int64

In [ ]:
##training model without Smoke

In [229]:
label_2 = le.fit_transform(labels_2)

In [121]:
X_train,X_test,y_train,y_test = train_test_split(scaled_temp_df_1,labels_2,stratify=labels_2)

In [97]:
param_grid_3 = {
    'C': [1],  
    'gamma': [1],
    'class_weight':['balanced']
}
 
svc_3 = SVC(probability=True)
 

In [98]:
GS_3 = GridSearchCV(svc_3,param_grid_3,n_jobs=-1,verbose=True)
GS_3.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.3s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1], 'class_weight': ['balanced'], 'gamma': [1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

{'C': 1, 'gamma': 1.5} --- 87

In [83]:
GS_3.best_params_

{'C': 1, 'gamma': 1}

In [99]:
GS_3.best_score_

0.7155631441746987

In [100]:
accuracy_score(y_test,GS_3.predict(X_test))

0.7332775919732442

In [ ]:
from sklearn.metrics import classification_report

In [101]:
print(classification_report(y_test,GS_3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.78      0.84      1049
           1       0.21      0.44      0.29       147

    accuracy                           0.73      1196
   macro avg       0.56      0.61      0.56      1196
weighted avg       0.82      0.73      0.77      1196



In [94]:
GS_3_predict_proba= GS_3.predict_proba(X_test)[:,0]

In [92]:
fpr,tpr,threshold_GS3= roc_curve(y_test,GS_3_predict_proba)

In [95]:
threshold_GS3

array([1.93861478, 0.93861478, 0.92622525, 0.92588721, 0.92335699,
       0.92333288, 0.92197105, 0.92192269, 0.92022176, 0.92005149,
       0.91951693, 0.91949089, 0.91776243, 0.91768806, 0.91496286,
       0.91480941, 0.91453878, 0.91453058, 0.91380746, 0.91379315,
       0.91324792, 0.91316358, 0.91153218, 0.91144433, 0.91049513,
       0.91048774, 0.90884912, 0.90880787, 0.90831006, 0.90820164,
       0.90800031, 0.90788533, 0.90773371, 0.90757513, 0.90682525,
       0.90670419, 0.90648416, 0.90644003, 0.90569097, 0.90567109,
       0.9051528 , 0.90504257, 0.90429008, 0.90427543, 0.90396536,
       0.90395091, 0.90317826, 0.90317151, 0.89999897, 0.89998257,
       0.89978266, 0.8997752 , 0.89972175, 0.89959875, 0.89912077,
       0.89906042, 0.8970165 , 0.8969848 , 0.89697645, 0.89691944,
       0.89551334, 0.89548684, 0.89451027, 0.89446352, 0.89431177,
       0.89430438, 0.89355431, 0.89349607, 0.89327929, 0.89325609,
       0.89215518, 0.8921529 , 0.89136962, 0.89128452, 0.89098

In [96]:
precision = []
recall = []
f1 = []
for i in threshold_GS3:
  y_pred_thresh = np.where(GS_3_predict_proba > i ,0,1)
  precision.append(precision_score(y_test,y_pred_thresh))
  recall.append(recall_score(y_test,y_pred_thresh))
  f1.append(f1_score(y_test,y_pred_thresh))
prec_recall_thresh_series = pd.concat([pd.Series(precision),pd.Series(recall),pd.Series(threshold_GS3),pd.Series(f1)],axis=1)
prec_recall_thresh_series.columns = ['precision','recall','threshold','f1']
print(prec_recall_thresh_series.sort_values(by='f1',ascending=False))

#0.5769

     precision    recall  threshold        f1
138   0.219880  0.496599   0.877431  0.304802
137   0.218289  0.503401   0.877680  0.304527
136   0.217647  0.503401   0.877704  0.303901
181   0.282353  0.326531   0.859779  0.302839
173   0.259804  0.360544   0.863846  0.301994
..         ...       ...        ...       ...
246   0.555556  0.034014   0.591271  0.064103
249   0.666667  0.027211   0.500000  0.052288
248   0.571429  0.027211   0.523886  0.051948
250   0.500000  0.006803   0.285086  0.013423
251   0.000000  0.000000   0.247718  0.000000

[252 rows x 4 columns]


In [230]:
sm = SMOTE()
Aug_scaled_features_GS_3,new_labels_GS_3=sm.fit_sample(scaled_temp_df_1,labels_2)
print(pd.value_counts(new_labels_GS_3))

Adult    4196
Youth    4196
dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [231]:
new_labels_GS_3 = le.fit_transform(new_labels_GS_3)
new_labels_GS_3

array([0, 0, 0, ..., 1, 1, 1])

In [232]:
X_train,X_test,y_train,y_test = train_test_split(Aug_scaled_features_GS_3,new_labels_GS_3,stratify=new_labels_GS_3)

In [233]:
param_grid_4 = {
    'C': [50],  
    'gamma': [2],
    
}
 
svc_3 = SVC()
 

In [234]:
y_train[0]

1

In [235]:
GS_4=GridSearchCV(svc_3,param_grid_4,n_jobs=-1)
GS_4.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1, param_grid={'C': [50], 'gamma': [2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [236]:
GS_4.best_params_

{'C': 50, 'gamma': 2}

In [237]:
GS_4.best_score_

0.912615432794847

In [238]:
print(classification_report(y_test,GS_4.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      1049
           1       0.90      0.98      0.93      1049

    accuracy                           0.93      2098
   macro avg       0.93      0.93      0.93      2098
weighted avg       0.93      0.93      0.93      2098



In [239]:
roc_auc_score(y_test,GS_4.predict(X_test))

0.9308865586272641

In [240]:
pickle.dump(GS_4,open('age.sav','wb'))
